tree census data exploration.  starting with 2015 census

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import geopandas as gpd


In [2]:
tree_df = pd.read_csv("new_york_tree_census_2015.csv")
tree_df.head()

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp
0,606945,305778,2016-06-28,10,0,OnCurb,Alive,Good,Fraxinus pennsylvanica,green ash,...,25,14,QN37,Kew Gardens Hills,4125700,New York,40.724339,-73.805180,1.038250e+06,203232.9417
1,160321,341273,2015-08-19,9,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,34,13,QN28,Jackson Heights,4030902,New York,40.756626,-73.894167,1.013571e+06,214953.6472
2,541347,325281,2015-12-30,7,0,OnCurb,Alive,Good,Pyrus calleryana,Callery pear,...,32,10,QN76,Baisley Park,4028800,New York,40.679777,-73.788463,1.042923e+06,187008.2671
3,613930,203822,2016-07-05,10,0,OnCurb,Alive,Good,Pyrus calleryana,Callery pear,...,46,22,BK31,Bay Ridge,3005000,New York,40.622743,-74.037543,9.738279e+05,166160.5847
4,18353,338911,2015-06-13,4,0,OnCurb,Alive,Good,Prunus virginiana,'Schubert' chokecherry,...,31,10,QN12,Hammels-Arverne-Edgemere,4095400,New York,40.596514,-73.797622,1.040452e+06,156667.5017


In [3]:
remove_values = ["Dead", "Stump"]
alive_df = tree_df[~tree_df["status"].isin(remove_values)]
alive_df.describe()

,tree_id,block_id,tree_dbh,stump_diam,zipcode,cb_num,borocode,cncldist,st_assem,st_senate,boro_ct,latitude,longitude,x_sp,y_sp
count,652173.000000,652173.000000,652173.000000,652173.0,652173.000000,652173.000000,652173.000000,652173.000000,652173.000000,652173.000000,6.521730e+05,652173.000000,652173.000000,6.521730e+05,652173.000000
mean,365689.637480,313611.213077,11.709485,0.0,10914.407958,343.738388,3.360979,29.992976,50.818103,20.620725,3.407340e+06,40.700991,-73.924587,1.005133e+06,194699.696220
std,208194.926983,114775.077889,8.634239,0.0,656.868397,115.808325,1.167572,14.343667,18.928937,7.378427,1.176612e+06,0.090268,0.123844,3.435720e+04,32886.140551
min,3.000000,100002.000000,0.000000,0.0,83.000000,101.000000,1.000000,1.000000,23.000000,10.000000,1.000201e+06,40.498466,-74.254965,9.133493e+05,120973.792200
25%,186886.000000,221491.000000,5.000000,0.0,10314.000000,303.000000,3.000000,19.000000,33.000000,14.000000,3.012000e+06,40.631725,-73.980973,9.895259e+05,169441.751400
50%,367204.000000,319886.000000,10.000000,0.0,11214.000000,402.000000,4.000000,30.000000,52.000000,21.000000,4.008100e+06,40.700287,-73.913449,1.008235e+06,194444.837500
75%,546632.000000,404665.000000,16.000000,0.0,11365.000000,412.000000,4.000000,43.000000,64.000000,25.000000,4.103300e+06,40.761999,-73.835219,1.029902e+06,216940.389000
max,722694.000000,999999.000000,425.000000,0.0,11697.000000,503.000000,5.000000,51.000000,87.000000,36.000000,5.032300e+06,40.912918,-73.700488,1.067248e+06,271894.092100


In [4]:
filtered_df = alive_df.drop(['created_at', 'stump_diam', 'status', 'user_type', 'sidewalk', 'guards', 'steward', 'st_assem', 
                             'st_senate', 'root_stone', 'root_grate', 'root_other', 'trunk_wire', 'trnk_light', 'trnk_other',
                             'brch_light', 'brch_shoe', 'brch_other'], axis=1)
filtered_df.head()

,tree_id,block_id,tree_dbh,curb_loc,health,spc_latin,spc_common,problems,address,zipcode,...,boroname,cncldist,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp
0,606945,305778,10,OnCurb,Good,Fraxinus pennsylvanica,green ash,Stones,76-046 164 STREET,11366,...,Queens,24,QN37,Kew Gardens Hills,4125700,New York,40.724339,-73.805180,1.038250e+06,203232.9417
1,160321,341273,9,OnCurb,Good,Gleditsia triacanthos var. inermis,honeylocust,BranchLights,72-020 32 AVENUE,11370,...,Queens,25,QN28,Jackson Heights,4030902,New York,40.756626,-73.894167,1.013571e+06,214953.6472
2,541347,325281,7,OnCurb,Good,Pyrus calleryana,Callery pear,BranchLights,153-026 119 AVENUE,11434,...,Queens,28,QN76,Baisley Park,4028800,New York,40.679777,-73.788463,1.042923e+06,187008.2671
3,613930,203822,10,OnCurb,Good,Pyrus calleryana,Callery pear,NaN,89 89 STREET,11209,...,Brooklyn,43,BK31,Bay Ridge,3005000,New York,40.622743,-74.037543,9.738279e+05,166160.5847
4,18353,338911,4,OnCurb,Good,Prunus virginiana,'Schubert' chokecherry,BranchLights,559 BEACH 68 STREET,11692,...,Queens,31,QN12,Hammels-Arverne-Edgemere,4095400,New York,40.596514,-73.797622,1.040452e+06,156667.5017


In [5]:
filtered_df.isnull().sum()

tree_id            0
block_id           0
tree_dbh           0
curb_loc           0
health             1
spc_latin          5
spc_common         5
problems      426329
address            0
zipcode            0
zip_city           0
cb_num             0
borocode           0
boroname           0
cncldist           0
nta                0
nta_name           0
boro_ct            0
state              0
latitude           0
longitude          0
x_sp               0
y_sp               0
dtype: int64

In [6]:
filtered_df.dropna(subset=["health", "spc_latin", "spc_common"], inplace=True)

In [7]:
filtered_df["problems"] = filtered_df["problems"].fillna("no problems")
filtered_df.head()

,tree_id,block_id,tree_dbh,curb_loc,health,spc_latin,spc_common,problems,address,zipcode,...,boroname,cncldist,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp
0,606945,305778,10,OnCurb,Good,Fraxinus pennsylvanica,green ash,Stones,76-046 164 STREET,11366,...,Queens,24,QN37,Kew Gardens Hills,4125700,New York,40.724339,-73.805180,1.038250e+06,203232.9417
1,160321,341273,9,OnCurb,Good,Gleditsia triacanthos var. inermis,honeylocust,BranchLights,72-020 32 AVENUE,11370,...,Queens,25,QN28,Jackson Heights,4030902,New York,40.756626,-73.894167,1.013571e+06,214953.6472
2,541347,325281,7,OnCurb,Good,Pyrus calleryana,Callery pear,BranchLights,153-026 119 AVENUE,11434,...,Queens,28,QN76,Baisley Park,4028800,New York,40.679777,-73.788463,1.042923e+06,187008.2671
3,613930,203822,10,OnCurb,Good,Pyrus calleryana,Callery pear,no problems,89 89 STREET,11209,...,Brooklyn,43,BK31,Bay Ridge,3005000,New York,40.622743,-74.037543,9.738279e+05,166160.5847
4,18353,338911,4,OnCurb,Good,Prunus virginiana,'Schubert' chokecherry,BranchLights,559 BEACH 68 STREET,11692,...,Queens,31,QN12,Hammels-Arverne-Edgemere,4095400,New York,40.596514,-73.797622,1.040452e+06,156667.5017


In [8]:
filtered_df.isnull().sum()

tree_id       0
block_id      0
tree_dbh      0
curb_loc      0
health        0
spc_latin     0
spc_common    0
problems      0
address       0
zipcode       0
zip_city      0
cb_num        0
borocode      0
boroname      0
cncldist      0
nta           0
nta_name      0
boro_ct       0
state         0
latitude      0
longitude     0
x_sp          0
y_sp          0
dtype: int64

In [9]:
filtered_df = filtered_df[filtered_df["zipcode"] != 83]
filtered_df["zipcode"].unique()

array([11366, 11370, 11434, 11209, 11692, 10031, 11207, 11219, 11421,
       10474, 11419, 10308, 10001, 10312, 11234, 10305, 11201, 11233,
       11411, 11220, 10009, 11372, 11432, 10314, 10461, 10309, 11367,
       11205, 10025, 11374, 11105, 11217, 10458, 11001, 10459, 11364,
       11379, 11691, 11004, 11358, 11238, 11213, 10470, 11356, 11103,
       11361, 10007, 11385, 11355, 11230, 10469, 10468, 10032, 11426,
       10304, 10463, 11206, 10462, 11216, 11208, 10465, 11375, 11429,
       11365, 11435, 11218, 10034, 11414, 11412, 11377, 11223, 11427,
       11204, 11357, 10022, 11215, 11428, 10036, 10456, 10024, 11221,
       10467, 10026, 11420, 11423, 10038, 11231, 11369, 11413, 10453,
       11210, 11436, 10307, 10475, 11229, 11363, 11040, 10003, 10457,
       11212, 11373, 11360, 11418, 11226, 10301, 10473, 10472, 11225,
       10302, 11422, 11354, 11236, 11106, 10013, 11214, 10452, 11378,
       10466, 10012, 10310, 11694, 10002, 11224, 10464, 10035, 11415,
       10306, 11222,

In [10]:
tree_df = filtered_df.drop(["block_id", "state", "cncldist", "nta", "boro_ct", "spc_latin", "address", "zip_city"], axis=1)
tree_df = tree_df.reset_index()
tree_df.head()

,index,tree_id,tree_dbh,curb_loc,health,spc_common,problems,zipcode,cb_num,borocode,boroname,nta_name,latitude,longitude,x_sp,y_sp
0,0,606945,10,OnCurb,Good,green ash,Stones,11366,408,4,Queens,Kew Gardens Hills,40.724339,-73.805180,1.038250e+06,203232.9417
1,1,160321,9,OnCurb,Good,honeylocust,BranchLights,11370,403,4,Queens,Jackson Heights,40.756626,-73.894167,1.013571e+06,214953.6472
2,2,541347,7,OnCurb,Good,Callery pear,BranchLights,11434,412,4,Queens,Baisley Park,40.679777,-73.788463,1.042923e+06,187008.2671
3,3,613930,10,OnCurb,Good,Callery pear,no problems,11209,310,3,Brooklyn,Bay Ridge,40.622743,-74.037543,9.738279e+05,166160.5847
4,4,18353,4,OnCurb,Good,'Schubert' chokecherry,BranchLights,11692,414,4,Queens,Hammels-Arverne-Edgemere,40.596514,-73.797622,1.040452e+06,156667.5017


In [11]:
tree_agg = tree_df.groupby("cb_num").agg({
    "tree_dbh": ["mean", "median", "max"],
    "tree_id": "count"
}).reset_index()
tree_agg.head()

cb_num  tree_dbh            tree_id
              mean median max   count
0    101  7.096212    6.0  88    2297
1    102  8.898200    8.0  33    4833
2    103  8.449352    7.0  32    4709
3    104  7.941163    8.0  38    4419
4    105  6.374504    6.0  21    2016

In [ ]:
tree_agg.to_csv("tree_summary_by_cb.csv", index=False)